In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import plotting

#plotly 
import plotly.offline as py
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
init_notebook_mode(connected=True)
import plotly.figure_factory as ff
import plotly.express as px

# filtering warnings
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
train=pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test=pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

In [ ]:
train.shape, test.shape


In [ ]:
k= pd.DataFrame()
k['missing count']= train.isnull().sum()
k.T

In [ ]:
sample=train.sample(frac=0.3)


In [ ]:
col=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19',
       'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28',
       'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37',
       'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46',
       'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55',
       'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64',
       'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73',
       'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82',
       'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91',
       'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100',
       'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108',
       'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116',
       'f117', 'f118' ]

In [ ]:
sample1=sample.dropna()


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
x=sample1[col]
y=sample1['claim']
X_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.3, random_state=10)

In [ ]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
lr=LogisticRegression(solver='liblinear')
lr.fit(X_train, y_train)

In [ ]:
y_pred=lr.predict(X_train)


In [ ]:
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

In [ ]:
y_test_pred=lr.predict(x_test)


In [ ]:
print("Recall Score:-", metrics.recall_score(y_test, y_test_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_test, y_test_pred))

In [ ]:
cfm=confusion_matrix(y_test, y_test_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cfm,
                               display_labels=lr.classes_)
disp.plot() 

In [ ]:
y_test_pred_prob=lr.predict_proba(x_test)[:,1]
y_test_pred_prob

from sklearn.metrics import roc_curve
metrics.roc_auc_score(y_test, y_test_pred_prob)

In [ ]:
fpr, tpr,thresholds=roc_curve(y_test,y_test_pred_prob)
plt.figure(figsize=(10,10))
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel("fpr (False Possitive rate)")
plt.ylabel("tpr-(True Positive rate)")
plt.title("ROC_AUC")
plt.show()

# Obs
* The ROC-AUC is worst than the Dummy model
* We need to improve our model.
* We need to remove som eof the features

In [ ]:
from scipy import stats


In [ ]:
cor_df=pd.DataFrame(columns=['r','p-value'])
for col in sample1:
   # print(col)
    if pd.api.types.is_numeric_dtype(sample1[col]):
        r, p=stats.pearsonr(sample1.claim, sample1[col])
        cor_df.loc[col]=[r, p]
cor_df

In [ ]:
cor_df.sort_values(by=['p-value'], ascending=False).head(20)


# Dropping these features

In [ ]:
col=['f28','f42','f16','f85','f29','f70','f88','f109','f53','f68', 'id', 'claim']

In [ ]:
x=sample1.drop(col, axis=1)
y=sample1['claim']
X_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.3, random_state=10)

In [ ]:
lr=LogisticRegression(solver='liblinear')
lr.fit(X_train, y_train)

In [ ]:
y_pred=lr.predict(X_train)


In [ ]:
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

In [ ]:
y_test_pred=lr.predict(x_test)


In [ ]:
print("Recall Score:-", metrics.recall_score(y_test, y_test_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_test, y_test_pred))

In [ ]:
y_test_pred_prob=lr.predict_proba(x_test)[:,1]
y_test_pred_prob

from sklearn.metrics import roc_curve
metrics.roc_auc_score(y_test, y_test_pred_prob)

In [ ]:
fpr, tpr,thresholds=roc_curve(y_test,y_test_pred_prob)
plt.figure(figsize=(10,10))
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel("fpr (False Possitive rate)")
plt.ylabel("tpr-(True Positive rate)")
plt.title("ROC_AUC")
plt.show()

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
error_rate=[]

for i in range(1,11):
    knn=KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred=knn.predict(x_test)
    error_rate.append(np.mean(pred!=y_test))
    
plt.figure(figsize=(15,10))
plt.plot(range(1,11), error_rate,marker='o', markersize=9)

In [ ]:
knn=KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train, y_train)

In [ ]:
y_pred=knn.predict(X_train)


In [ ]:
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

In [ ]:
y_test_pred=knn.predict(x_test)


In [ ]:
print("Recall Score:-", metrics.recall_score(y_test, y_test_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_test, y_test_pred))

In [ ]:
y_test_pred_prob=knn.predict_proba(x_test)[:,1]
y_test_pred_prob

from sklearn.metrics import roc_curve
metrics.roc_auc_score(y_test, y_test_pred_prob)

In [ ]:
fpr, tpr,thresholds=roc_curve(y_test,y_test_pred_prob)
plt.figure(figsize=(10,10))
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel("fpr (False Possitive rate)")
plt.ylabel("tpr-(True Positive rate)")
plt.title

# Multiple Imputation

In [ ]:
from sklearn.impute import SimpleImputer


In [ ]:
my_imputer = SimpleImputer()


In [ ]:
data_with_imputed_values = my_imputer.fit_transform(sample)


In [ ]:
sample2=pd.DataFrame(data_with_imputed_values)

In [ ]:
sample2.head()

In [ ]:
sample2.columns=sample1.columns

In [ ]:
sample1.head()

In [ ]:
sample2.head()

In [ ]:
k= pd.DataFrame()
k['missing count']= sample2.isnull().sum()
k.T


In [ ]:
col=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19',
       'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28',
       'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37',
       'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46',
       'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55',
       'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64',
       'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73',
       'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82',
       'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91',
       'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100',
       'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108',
       'f109', 'f110']

# Logistic regresson

In [ ]:
x=sample2[col]
y=sample2['claim']
X_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.3, random_state=10)

In [ ]:
lr=LogisticRegression(solver='liblinear')
lr.fit(X_train, y_train)

In [ ]:
y_pred=lr.predict(X_train)


In [ ]:
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

In [ ]:
y_test_pred=lr.predict(x_test)


In [ ]:
print("Recall Score:-", metrics.recall_score(y_test, y_test_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_test, y_test_pred))

In [ ]:
y_test_pred_prob=lr.predict_proba(x_test)[:,1]
y_test_pred_prob

from sklearn.metrics import roc_curve
metrics.roc_auc_score(y_test, y_test_pred_prob)

In [ ]:
fpr, tpr,thresholds=roc_curve(y_test,y_test_pred_prob)
plt.figure(figsize=(10,10))
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel("fpr (False Possitive rate)")
plt.ylabel("tpr-(True Positive rate)")
plt.title("ROC_AUC")
plt.show()

# Standarization

In [ ]:
#Standardization
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)

In [ ]:

X_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.3, random_state=10)

In [ ]:
lr=LogisticRegression(solver='liblinear')
lr.fit(X_train, y_train)

In [ ]:
y_pred=lr.predict(X_train)


In [ ]:
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

In [ ]:
y_test_pred=lr.predict(x_test)


In [ ]:
print("Recall Score:-", metrics.recall_score(y_test, y_test_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_test, y_test_pred))

In [ ]:
y_test_pred_prob=lr.predict_proba(x_test)[:,1]
y_test_pred_prob

from sklearn.metrics import roc_curve
metrics.roc_auc_score(y_test, y_test_pred_prob)

In [ ]:
fpr, tpr,thresholds=roc_curve(y_test,y_test_pred_prob)
plt.figure(figsize=(10,10))
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel("fpr (False Possitive rate)")
plt.ylabel("tpr-(True Positive rate)")
plt.title("ROC_AUC")
plt.show()

# Work with Whole dataset

In [ ]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
data = my_imputer.fit_transform(train)


In [ ]:
df=pd.DataFrame(data)

In [ ]:
df.columns=sample1.columns

In [ ]:
X=df.drop(col, axis=1)
y=df['claim']

In [ ]:
#Standardization
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=sc.fit_transform(X)

In [ ]:

X_train, x_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
lr=LogisticRegression(solver='liblinear')
lr.fit(X_train, y_train)

In [ ]:
y_pred=lr.predict(X_train)


In [ ]:
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

In [ ]:
y_test_pred=lr.predict(x_test)


In [ ]:
print("Recall Score:-", metrics.recall_score(y_test, y_test_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_test, y_test_pred))

In [ ]:
y_test_pred_prob=lr.predict_proba(x_test)[:,1]
y_test_pred_prob

from sklearn.metrics import roc_curve
metrics.roc_auc_score(y_test, y_test_pred_prob)

In [ ]:
fpr, tpr,thresholds=roc_curve(y_test,y_test_pred_prob)
plt.figure(figsize=(10,10))
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel("fpr (False Possitive rate)")
plt.ylabel("tpr-(True Positive rate)")
plt.title("ROC_AUC")
plt.show()

# Obs
* Standarization has effect on logistic regression 
* The ROC score is much better 

In [ ]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
data_test = my_imputer.fit_transform(test)

In [ ]:
col

In [ ]:
test=pd.DataFrame(data_test)

In [ ]:
test.head()

In [ ]:
sample.columns.values

In [ ]:
col=['id','f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19',
       'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28',
       'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37',
       'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46',
       'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55',
       'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64',
       'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73',
       'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82',
       'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91',
       'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100',
       'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108',
       'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116',
       'f117', 'f118']

In [ ]:
test.columns=col

In [ ]:
col1=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19',
       'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28',
       'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37',
       'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46',
       'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55',
       'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64',
       'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73',
       'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82',
       'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91',
       'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100',
       'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108',
       'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116',
       'f117', 'f118']

In [ ]:
x_test=sc.fit_transform(test[col1])

In [ ]:
x_test